# Diamond Best Blast Hits (BBH)

In [5]:
import subprocess
import io
import gzip
import os

import pandas as pd

from collections import OrderedDict
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Generate the diamonds DBs

In [43]:
file_df = pd.read_csv("diamondBBH_files.tsv", index_col=None, sep="\t", header=0)
file_df

,Species,Genotype,File-name
0,Zea mays,CML247,Zm-CML247-REFERENCE-NAM-1.0_Zm00023ab.1.protei...
1,Zea mays,CML69,Zm-CML69-REFERENCE-NAM-1.0_Zm00020ab.1.protein...
2,Zea mays,B73v5,Zm-B73-REFERENCE-NAM-5.0_Zm00001eb.1.protein.c...
3,Zea mays,CML333,Zm-CML333-REFERENCE-NAM-1.0_Zm00026ab.1.protei...
4,Zea mays,Ki3,Zm-Ki3-REFERENCE-NAM-1.0_Zm00029ab.1.protein.c...
5,Sorghum bicolor,Sbicolor,Sbicolor_454_v3.1.1.protein_primaryTranscriptO...
6,Zea mays,CML277,Zm-CML277-REFERENCE-NAM-1.0_Zm00024ab.1.protei...
7,Zea mays,Oh7B,Zm-Oh7B-REFERENCE-NAM-1.0_Zm00038ab.1.protein....
8,Zea mays,CML52,Zm-CML52-REFERENCE-NAM-1.0_Zm00019ab.1.protein...
9,Zea mays,Il14H,Zm-Il14H-REFERENCE-NAM-1.0_Zm00028ab.1.protein...


In [13]:
# Generate a diamon database for the reference proteome
ref_name = "B73v4"
ref_file = df[df["Genotype"]==ref_name]["File-name"].values[0]
os.system("gunzip --keep --stdout files/{0} | diamond makedb --in /dev/stdin --db dbs/{1}".format(ref_file, ref_name))

diamond v0.9.21.122 | by Benjamin Buchfink <buchfink@gmail.com>
Licensed under the GNU AGPL <https://www.gnu.org/licenses/agpl.txt>
Check http://github.com/bbuchfink/diamond for updates.

#CPU threads: 32
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database file: /dev/stdin
Opening the database file...  [0.003343s]
Loading sequences...  [0.212997s]
Masking sequences...  [0.225716s]
Writing sequences...  [0.026307s]
Hashing sequences...  [0.006229s]
Loading sequences...  [1.7e-05s]
Writing trailer...  [0.000925s]
Closing the input file...  [2.1e-05s]
Closing the database file...  [0.050217s]
Database hash = 4db8aae837a713e75aa1499806ba7c22
Processed 39498 sequences, 15474351 letters.
Total time = 0.525989s


0

In [44]:
genotype_dict = file_df[["Genotype", "File-name"]].set_index('Genotype').to_dict()["File-name"]
genotype_dict

{'CML247': 'Zm-CML247-REFERENCE-NAM-1.0_Zm00023ab.1.protein.canonical.fa.gz',
 'CML69': 'Zm-CML69-REFERENCE-NAM-1.0_Zm00020ab.1.protein.canonical.fa.gz',
 'B73v5': 'Zm-B73-REFERENCE-NAM-5.0_Zm00001eb.1.protein.canonical.fa.gz',
 'CML333': 'Zm-CML333-REFERENCE-NAM-1.0_Zm00026ab.1.protein.canonical.fa.gz',
 'Ki3': 'Zm-Ki3-REFERENCE-NAM-1.0_Zm00029ab.1.protein.canonical.fa.gz',
 'Sbicolor': 'Sbicolor_454_v3.1.1.protein_primaryTranscriptOnly.locus.fa.gz',
 'CML277': 'Zm-CML277-REFERENCE-NAM-1.0_Zm00024ab.1.protein.canonical.fa.gz',
 'Oh7B': 'Zm-Oh7B-REFERENCE-NAM-1.0_Zm00038ab.1.protein.canonical.fa.gz',
 'CML52': 'Zm-CML52-REFERENCE-NAM-1.0_Zm00019ab.1.protein.canonical.fa.gz',
 'Il14H': 'Zm-Il14H-REFERENCE-NAM-1.0_Zm00028ab.1.protein.canonical.fa.gz',
 'Sitalica': 'Sitalica_312_v2.2.protein_primaryTranscriptOnly.locus.fa.gz',
 'P39': 'Zm-P39-REFERENCE-NAM-1.0_Zm00040ab.1.protein.canonical.fa.gz',
 'M37W': 'Zm-M37W-REFERENCE-NAM-1.0_Zm00032ab.1.protein.canonical.fa.gz',
 'Oh43': 'Zm-Oh43-

In [45]:
df = pd.DataFrame()
reference = "B73v4"
comb_len = 0
#genotype_dict = {'B73v4': 'Zmays_493_RefGen_V4.protein_primaryTranscriptOnly.locus.fa.gz',}
genotype_dict = file_df[["Genotype", "File-name"]].set_index('Genotype').to_dict()["File-name"]
for qname in genotype_dict.keys():
        # /dev/stdout only works if provided with permission: sudo chown -R $USER /dev
        results = pd.read_csv(io.StringIO(subprocess.check_output("gunzip --keep --stdout files/{0} | \
                   diamond blastp --quiet -p 32 -d dbs/{1} -q /dev/stdin -o /dev/stdout \
                   --more-sensitive --min-score 300 --outfmt 6 sseqid qseqid bitscore".format(genotype_dict[qname], reference),
                   shell=True, text=True)), sep="\t", header=None) # --evalue  1e-100
        results["subject"] = "B73v4"
        results["query"] = qname
        df = df.append(results)
        comb_len += len(df)
        print("Finished:", qname, "and their length is:", comb_len)

Finished: CML247 and their length is: 117548
Finished: CML69 and their length is: 350355
Finished: B73v5 and their length is: 700244
Finished: CML333 and their length is: 1166617
Finished: Ki3 and their length is: 1750321
Finished: Sbicolor and their length is: 2432594
Finished: CML277 and their length is: 3231691
Finished: Oh7B and their length is: 4148607
Finished: CML52 and their length is: 5184943
Finished: Il14H and their length is: 6338358
Finished: Sitalica and their length is: 7595117
Finished: P39 and their length is: 8969350
Finished: M37W and their length is: 10462269
Finished: Oh43 and their length is: 12072772
Finished: HP301 and their length is: 13800381
Finished: Tzi8 and their length is: 15648058
Finished: Osativa and their length is: 17588849
Finished: Mo18W and their length is: 19645887
Finished: Ki11 and their length is: 21820733
Finished: M162W and their length is: 24113547
Finished: Col-0 and their length is: 26472425
Finished: B73v4 and their length is: 28980357
F

In [48]:
# Final steps before saving file as tsv to save on some space
df.columns = ["subject_id", "query_id", "bitscore", "subject", "query"]
df['bitscore'] = df['bitscore'].astype(int)
df.to_csv("diamond_ref.tsv", sep="\t", index=False)